In [24]:
try:
    import tangelo
except ModuleNotFoundError:
    !pip install git+https://github.com/goodchemistryco/Tangelo.git@develop  --quiet
    !pip install pyscf
    !pip install qulacs qiskit qiskit-aer  

In [2]:
from tangelo import SecondQuantizedMolecule

LiH = [('Li', (0, 0, 0)),('H', (0, 0, 1.5949))] #these triples refer to coordinates
# the 1.5949 is experimentally determined to be the space betwn the elements
#see here for example https://cccbdb.nist.gov/expgeom2x.asp?casno=1333740
#use frozen_orbitals = None to set everything active for calculations
mol_LiH = SecondQuantizedMolecule(LiH, q=0, spin=0, basis="sto-3g")

# triples can be made using spin #, abs diff between alpha and beta electrons
#alpha = spin up, beta = spin-down
#spin = alpha - beta electrons, 2 electrons make alpha(it seems) beta is single 
#electron occupying an orbital
mol_LiH_t = SecondQuantizedMolecule(LiH, q=0, spin=2, basis="sto-3g")


In [3]:
#Recieve visualization of the molecule for verification

#there should be 6 spin orbitals, which can hold a max of 2 electrons
#2 are occupied, with one active and one frozen and should have 4
#total electrons

# Note, frozen orbitals mean that some electrons are treated as fixed
# so if one orbital is frozen => only 2 electrons get qbits as opposed to 
#others in the frozen orbital

#mo_occ parameter tells use where our electrons are!
print(mol_LiH) # 2, 2 split between 0 and 1 oribtals
print(mol_LiH_t) # 2,1,1 splits between 0
print(mol_LiH.n_mos, mol_LiH.n_sos)

SecondQuantizedMolecule(xyz=[('Li', (0.0, 0.0, 0.0)), ('H', (0.0, 0.0, 1.5949))], q=0, spin=0, solver=<tangelo.toolboxes.molecular_computation.integral_solver_pyscf.IntegralSolverPySCF object at 0x796190704190>, n_atoms=2, n_electrons=4, basis='sto-3g', ecp={}, symmetry=False, uhf=False, mf_energy=-7.862026959394135, mo_energies=array([-2.34864417, -0.28570476,  0.07826185,  0.16393842,  0.16393842,
        0.54912925]), mo_occ=array([2., 2., 0., 0., 0., 0.]), n_mos=6, n_sos=12, active_occupied=[1], frozen_occupied=[0], active_virtual=[2, 3, 4, 5], frozen_virtual=[])
SecondQuantizedMolecule(xyz=[('Li', (0.0, 0.0, 0.0)), ('H', (0.0, 0.0, 1.5949))], q=0, spin=2, solver=<tangelo.toolboxes.molecular_computation.integral_solver_pyscf.IntegralSolverPySCF object at 0x796130ba7bd0>, n_atoms=2, n_electrons=4, basis='sto-3g', ecp={}, symmetry=False, uhf=False, mf_energy=-7.766133365335711, mo_energies=NPArrayWithTag([-2.34705371, -0.26589778,  0.01337911,  0.16088823,
                 0.16088823

In [4]:
from tangelo.algorithms import FCISolver, CCSDSolver
#classical energy calculations using FCI and CCSD
#FCI = bad scale good acc, CCSD = good scale worse acc
fci_solver = FCISolver(mol_LiH_t)
fci_energy = fci_solver.simulate()

print(f"LiH FCI Groud State Energy: {fci_energy}")

ccsd_solver = CCSDSolver(mol_LiH_t)
ccsd_energy = ccsd_solver.simulate()
print(f"LiH CCSD Groud State Energy: {ccsd_energy}")
print(f"Percent difference of FCI and CCSD: {100*(fci_energy-ccsd_energy)/fci_energy}%")

LiH FCI Groud State Energy: -7.766133365335712
LiH CCSD Groud State Energy: -7.766133365335711
Percent difference of FCI and CCSD: 1.1436558940186721e-14%


In [5]:
#Implement a VQE Energy solver
from tangelo.algorithms import VQESolver

#100 cycles for speed's sake
vqe_options = {"molecule": mol_LiH, "verbose": True}

vqe_solver = VQESolver(vqe_options)
vqe_solver.build()
vqe_energy = vqe_solver.simulate()

print(f"VQE Ground State Energy: {vqe_energy}")
print(f"Percent difference of FCI and VQE: {100*(fci_energy-vqe_energy)/fci_energy}%")

"""
VQE Ground State Energy: -7.882390470334529
Percent difference of FCI and VQE: 0.00016416314034363796%
"""

	Energy = -7.8790982 
	Energy = -7.8790981 
	Energy = -7.8790982 
	Energy = -7.8790982 
	Energy = -7.8790982 
	Energy = -7.8790983 
	Energy = -7.8790983 
	Energy = -7.8790983 
	Energy = -7.8791000 
	Energy = -7.8790982 
	Energy = -7.8790982 
	Energy = -7.8790976 
	Energy = -7.8790982 
	Energy = -7.8790982 
	Energy = -7.8790982 
	Energy = -7.7757195 
	Energy = -7.8814177 
	Energy = -7.8814176 
	Energy = -7.8814177 
	Energy = -7.8814177 
	Energy = -7.8814176 
	Energy = -7.8814177 
	Energy = -7.8814177 
	Energy = -7.8814177 
	Energy = -7.8814175 
	Energy = -7.8814177 
	Energy = -7.8814177 
	Energy = -7.8814172 
	Energy = -7.8814177 
	Energy = -7.8814177 
	Energy = -7.8814177 
	Energy = -7.8806231 
	Energy = -7.8818623 
	Energy = -7.8818621 
	Energy = -7.8818623 
	Energy = -7.8818623 
	Energy = -7.8818622 
	Energy = -7.8818624 
	Energy = -7.8818623 
	Energy = -7.8818623 
	Energy = -7.8818622 
	Energy = -7.8818623 
	Energy = -7.8818623 
	Energy = -7.8818623 
	Energy = -7.8818623 
	Energy = 

'\nVQE Ground State Energy: -7.882390470334529\nPercent difference of FCI and VQE: 0.00016416314034363796%\n'

In [7]:
from pprint import pprint
pprint(vars(vqe_solver))

{'ansatz': <tangelo.toolboxes.ansatz_generator.uccsd.UCCSD object at 0x7f312b304ad0>,
 'ansatz_options': {},
 'backend': <tangelo.linq.target.target_qulacs.QulacsSimulator object at 0x7f30f91a8190>,
 'backend_options': {'n_shots': None, 'noise_model': None, 'target': None},
 'builtin_ansatze': {<BuiltInAnsatze.ILC: <class 'tangelo.toolboxes.ansatz_generator.ilc.ILC'>>,
                     <BuiltInAnsatze.QMF: <class 'tangelo.toolboxes.ansatz_generator.qmf.QMF'>>,
                     <BuiltInAnsatze.UCCGD: <class 'tangelo.toolboxes.ansatz_generator.uccgd.UCCGD'>>,
                     <BuiltInAnsatze.UCC3: <tangelo.toolboxes.ansatz_generator.rucc.RUCC object at 0x7f312bf9d490>>,
                     <BuiltInAnsatze.HEA: <class 'tangelo.toolboxes.ansatz_generator.hea.HEA'>>,
                     <BuiltInAnsatze.VSQS: <class 'tangelo.toolboxes.ansatz_generator.vsqs.VSQS'>>,
                     <BuiltInAnsatze.UCCSD: <class 'tangelo.toolboxes.ansatz_generator.uccsd.UCCSD'>>,
           

In [36]:
#tangelo implements trotter suzuki time evolution
#we can use that module to do things, but we need to find
#the qubit hamiltonian of our molecule
#Steps: get hamiltonian, build Trotter circuit, simulate circuit in diff ways
#(noisy, noiseless, quantum machine hardware, etc.)
from tangelo.toolboxes.unitary_generator.trotter_suzuki import TrotterSuzukiUnitary

NUM_STEPS = 10

LiH_gnd_hamiltonian = vqe_solver.qubit_hamiltonian
ts_model = TrotterSuzukiUnitary(qubit_hamiltonian=LiH_gnd_hamiltonian)
ts_circuit = ts_model.build_circuit(NUM_STEPS)



In [11]:
#we now need to do a simulation of our trotter suzuki circuit
print(ts_circuit)
ts_circuit.draw()


Circuit object. Size 3965 

RZ        target : [0]   parameter : -17.294535534699687
RX        target : [0]   parameter : 1.5707963267948966
RX        target : [2]   parameter : 1.5707963267948966
CNOT      target : [1]   control : [0]   
CNOT      target : [2]   control : [1]   
RZ        target : [2]   parameter : 12.203436533709294
CNOT      target : [2]   control : [1]   
CNOT      target : [1]   control : [0]   
RX        target : [2]   parameter : -1.5707963267948966
RX        target : [0]   parameter : -1.5707963267948966
H         target : [0]   
H         target : [2]   
CNOT      target : [1]   control : [0]   
CNOT      target : [2]   control : [1]   
RZ        target : [2]   parameter : 12.203436533709294
CNOT      target : [2]   control : [1]   
CNOT      target : [1]   control : [0]   
H         target : [2]   
H         target : [0]   
RX        target : [0]   parameter : 1.5707963267948966
RX        target : [8]   parameter : 1.5707963267948966
CNOT      target : [1]   

KeyboardInterrupt: 

In [37]:
# Try to run circuit simulations with/without noises on backend
from tangelo.linq import get_backend, ONE_QUBIT_GATES, TWO_QUBIT_GATES
from tangelo.linq.noisy_simulation import NoiseModel

# Get the optimal circuit
optimal_circuit = vqe_solver.optimal_circuit

# Create a noise model to be used
nm = NoiseModel()
for g in ONE_QUBIT_GATES:
    nm.add_quantum_error(g, 'pauli', [1/4]*3)
nm.add_quantum_error('CNOT', 'pauli', [1., 0., 0.])
nm.add_quantum_error('CNOT', 'depol', 0.72)

# Get backends with noisy or noiseless 
#noisy_backend = get_backend(target='qiskit', n_shots=10**6, noise_model=nm)
noisless_backend = get_backend(target='qiskit')

# Simulate
#print(noisy_backend.simulate(optimal_circuit))
print(noisless_backend.simulate(ts_circuit))



({'0000000000': 0.9999999999999825}, None)


In [ ]:
# Try different Mappings
from tangelo.algorithms import BuiltInAnsatze as Ansatze
# VQE-UCCSD on LiH with different qubit mappings
for qm in ['jw', 'bk', 'scbk']:
    vqe1_options = {"molecule": mol_LiH, "ansatz": Ansatze.UCCSD, "qubit_mapping": qm}
    vqe1_solver = VQESolver(vqe1_options)
    vqe1_solver.build()
    vqe1_solver.simulate()
    print(f"VQE Ground State Energy: {vqe1_solver.optimal_energy}")
    print(f"Percent difference of FCI and VQE: {100*(fci_energy-vqe1_solver.optimal_energy)/fci_energy}%")
    print("\n", vqe1_solver.get_resources(), "\n")

In [12]:
# Try different Ansatz ('jw' as the mapping; HEA isn't compatible with 'jw')
from tangelo.algorithms import BuiltInAnsatze as Ansatze
for az in [Ansatze.UCCSD, Ansatze.ILC, Ansatze.QCC, Ansatze.QMF]:
    vqe2_options = {"molecule": mol_LiH, "ansatz": az, "up_then_down": True, "qubit_mapping": 'jw'}
    vqe2_solver = VQESolver(vqe2_options)
    vqe2_solver.build()
    vqe2_solver.simulate()
    print(f"VQE Ground State Energy: {vqe2_solver.optimal_energy}")
    print(f"Percent difference of FCI and VQE: {100*(fci_energy-vqe2_solver.optimal_energy)/fci_energy}%")
    print("\n", vqe2_solver.get_resources(), "\n")


VQE Ground State Energy: -7.882174298860317
Percent difference of FCI and VQE: 2.1465404420527024e-05%

 {'qubit_hamiltonian_terms': 276, 'circuit_width': 10, 'circuit_depth': 1985, 'circuit_2qubit_gates': 1616, 'circuit_var_gates': 144, 'vqe_variational_parameters': 14} 

VQE Ground State Energy: -7.881746411660311
Percent difference of FCI and VQE: 0.005450006970930077%

 {'qubit_hamiltonian_terms': 276, 'circuit_width': 10, 'circuit_depth': 133, 'circuit_2qubit_gates': 98, 'circuit_var_gates': 31, 'vqe_variational_parameters': 26} 

VQE Ground State Energy: -7.881490166421676
Percent difference of FCI and VQE: 0.008700952381614063%

 {'qubit_hamiltonian_terms': 276, 'circuit_width': 10, 'circuit_depth': 56, 'circuit_2qubit_gates': 36, 'circuit_var_gates': 26, 'vqe_variational_parameters': 26} 

VQE Ground State Energy: -7.862026959394126
Percent difference of FCI and VQE: 0.25562777880954346%

 {'qubit_hamiltonian_terms': 276, 'circuit_width': 10, 'circuit_depth': 2, 'circuit_2qubit

In [13]:
# Try different Ansatz ('bk' as map)
for az in [Ansatze.UCCSD, Ansatze.ILC, Ansatze.QCC, Ansatze.QMF]:
    vqe3_options = {"molecule": mol_LiH, "ansatz": az, "up_then_down": True, "qubit_mapping": 'bk'}
    vqe3_solver = VQESolver(vqe3_options)
    vqe3_solver.build()
    vqe3_solver.simulate()
    print(f"VQE Ground State Energy: {vqe3_solver.optimal_energy}")
    print(f"Percent difference of FCI and VQE: {100*(fci_energy-vqe3_solver.optimal_energy)/fci_energy}%")
    print("\n", vqe3_solver.get_resources(), "\n")

# 'bk' mapping seems to be a little better for the default setting of UCCSD, but for ILC and QCC it depends on whether "up_then_down" is False.
# ILC and QCC provide the best balance between accuracy and circuit complexity compared to other ansatzes no matter the mapping used.

VQE Ground State Energy: -7.882172112674606
Percent difference of FCI and VQE: 4.920121891179857e-05%

 {'qubit_hamiltonian_terms': 276, 'circuit_width': 10, 'circuit_depth': 1834, 'circuit_2qubit_gates': 1442, 'circuit_var_gates': 144, 'vqe_variational_parameters': 14} 

VQE Ground State Energy: -7.8817464116602896
Percent difference of FCI and VQE: 0.0054500069712005135%

 {'qubit_hamiltonian_terms': 276, 'circuit_width': 10, 'circuit_depth': 127, 'circuit_2qubit_gates': 92, 'circuit_var_gates': 31, 'vqe_variational_parameters': 26} 

VQE Ground State Energy: -7.881490166422676
Percent difference of FCI and VQE: 0.008700952368926084%

 {'qubit_hamiltonian_terms': 276, 'circuit_width': 10, 'circuit_depth': 64, 'circuit_2qubit_gates': 44, 'circuit_var_gates': 26, 'vqe_variational_parameters': 26} 

VQE Ground State Energy: -7.862026959394125
Percent difference of FCI and VQE: 0.25562777880955473%

 {'qubit_hamiltonian_terms': 276, 'circuit_width': 10, 'circuit_depth': 2, 'circuit_2qubi

In [15]:
# Try different Ansatz ('scbk' as the mapping, which strictly requires "up_then_down" = True)
from tangelo.algorithms import BuiltInAnsatze as Ansatze
for az in [Ansatze.UCCSD, Ansatze.ILC, Ansatze.QCC, Ansatze.QMF]:
    vqe3_options = {"molecule": mol_LiH, "ansatz": az, "up_then_down": True, "qubit_mapping": 'scbk'}
    vqe3_solver = VQESolver(vqe3_options)
    vqe3_solver.build()
    vqe3_solver.simulate()
    print(f"VQE Ground State Energy: {vqe3_solver.optimal_energy}")
    print(f"Percent difference of FCI and VQE: {100*(fci_energy-vqe3_solver.optimal_energy)/fci_energy}%")
    print("\n", vqe3_solver.get_resources(), "\n")

## We can conclude that the specific mapping should be tuned for each circuit, but the choice of Ansatze matters the most.
## UCCSD is the most accurate (10^-5 %) one but produces the most complex circuit.
## ILC and QCC have a good balance (10^-3 %), though each might favor different mappings depending on the circuit.
## VQE produces the least complex circuit with a slightly higher relative relative error (10^-1 %).
    
## TODO: Quantify the accuracy and circuit simplicity we aim for.

VQE Ground State Energy: -7.882170877578321
Percent difference of FCI and VQE: 6.487070265754792e-05%

 {'qubit_hamiltonian_terms': 276, 'circuit_width': 8, 'circuit_depth': 1660, 'circuit_2qubit_gates': 1276, 'circuit_var_gates': 144, 'vqe_variational_parameters': 14} 

VQE Ground State Energy: -7.881746411660279
Percent difference of FCI and VQE: 0.005450006971335732%

 {'qubit_hamiltonian_terms': 276, 'circuit_width': 8, 'circuit_depth': 129, 'circuit_2qubit_gates': 94, 'circuit_var_gates': 27, 'vqe_variational_parameters': 22} 

VQE Ground State Energy: -7.881490166423446
Percent difference of FCI and VQE: 0.008700952359156565%

 {'qubit_hamiltonian_terms': 276, 'circuit_width': 8, 'circuit_depth': 64, 'circuit_2qubit_gates': 44, 'circuit_var_gates': 22, 'vqe_variational_parameters': 22} 

VQE Ground State Energy: -7.862026959394125
Percent difference of FCI and VQE: 0.25562777880955473%

 {'qubit_hamiltonian_terms': 276, 'circuit_width': 8, 'circuit_depth': 2, 'circuit_2qubit_gate